In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
# @title Default title text
import json
import random
from groq import Groq
import nltk
import re
from scipy.optimize import linear_sum_assignment
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Ensure you have the necessary NLTK data files
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def read_claims_from_json(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except json.JSONDecodeError:
        print(f"Error decoding JSON from file: {file_path}")
        return []

# Load your dataset from JSON files
claims_file_1 = '/content/Modified_true_claims.json'
claims_file_2 = '/content/last_fake_claims.json'

claims_data_1 = read_claims_from_json(claims_file_1)
claims_data_2 = read_claims_from_json(claims_file_2)

# Combine claims from both files
combined_claims = claims_data_1 + claims_data_2
random.shuffle(combined_claims)

In [ ]:
# Initialize Groq API client
client = Groq(api_key="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")


import re

def convert_to_json(llm_response):
    responseQuestions = []
    questions_llm = re.findall(r'\{(.*?)\}', llm_response, re.DOTALL)

    for question in questions_llm:
        question_json = {}

        q = re.findall(r'[\"\']question[\"\']:\s*[\"\']?(.*?)[\"\'],', question, re.DOTALL)
        if q:
            question_json['question'] = q[0].strip()
        else:
            continue

        a = re.findall(r'[\"\']answer[\"\']:\s*[\"\']?(.*?)[\"\'],', question, re.DOTALL)
        if a:
            question_json['answer'] = a[0].strip()

        at = re.findall(r'[\"\']answerType[\"\']:\s*[\"\']?(.*?)[\"\']', question, re.DOTALL)
        if at:
            question_json['answerType'] = at[0].strip()

        responseQuestions.append(question_json)

    return responseQuestions



def get_response_llm(claim, questions):
  prompt = f"{questions} \n Claim: {claim}. Provide me only with similar questions relevant for the claim given in similar json format. Provide boolean, Abstractive, Extractive questions and answer pairs in a single list json. Do not leave question and answer pair empty"

  try:
      response = client.chat.completions.create(
          messages=[
              {
                  "role": "user",
                  "content": prompt,
              }
          ],
          model="gemma2-9b-it",
          max_tokens=4000,
      )
      response = response.choices[0].message.content.strip()
      response_json = convert_to_json(response)
      return response_json
  except Exception as e:
      print(f"Error processing claim '{claim}': {e}, {response}")
      print("-*"*50)
      return ""

In [ ]:
new_data = []

for data in combined_claims[:5010]:
  questions_prompt = []
  claim = data["claim"]
  for question in combined_claims[0]["questions"]:
    q = {"question": question["question"]}
    answer = question["answers"][0]
    q['answer'] = answer['answer']
    q['answerType'] = answer['answer_type']
    questions_prompt.append(q)
  response = get_response_llm(claim, questions_prompt)
  data = {'claim': claim, 'generatedQuestions': response, 'goldQuestions': questions_prompt}
  new_data.append(data)

In [ ]:
print(len([1 for n in new_data if n['generatedQuestions'] == []]))
new_data

0


[{'claim': 'RSS activists are harassing Muslim women.',
  'generatedQuestions': [{'question': 'Is there evidence of RSS activists harassing Muslim women in India?',
    'answer': 'No.',
    'answerType': 'Boolean'},
   {'question': 'Are there any reliable sources that confirm RSS activists are targeting Muslim women?',
    'answer': 'No, claims of RSS activists targeting Muslim women are often based on misinformation and misrepresented incidents.',
    'answerType': 'Abstractive'},
   {'question': 'What are the facts about the video showing Muslim women being harassed?',
    'answer': 'The video showing Muslim women being harassed originated in Sri Lanka and was falsely attributed to RSS activists in India.',
    'answerType': 'Extractive'},
   {'question': 'Has the RSS organization been linked to violence against Muslim women?',
    'answer': 'There is no credible evidence to support claims that the RSS organization has been linked to violence against Muslim women.',
    'answerType':

In [ ]:
hungarian_meteor_scores = []

def calculate_meteor_scores(generated_sequences, reference_sequences):
    """
    Calculate the METEOR scores between generated sequences and reference sequences.

    :param generated_sequences: List of generated sequences (list of strings)
    :param reference_sequences: List of reference sequences (list of strings)
    :return: A 2D list of METEOR scores where scores[i][j] is the score between generated_sequences[i] and reference_sequences[j]
    """
    scores = []
    for gen_seq in generated_sequences:
        row = []
        for ref_seq in reference_sequences:
            score = meteor_score([ref_seq], gen_seq)
            row.append(score)
        scores.append(row)
    return scores

def hungarian_meteor(generated_sequences, reference_sequences):
    """
    Apply the Hungarian Algorithm to find the optimal matching between generated sequences and reference sequences.

    :param generated_sequences: List of generated sequences (list of strings)
    :param reference_sequences: List of reference sequences (list of strings)
    :return: A list of tuples where each tuple contains the index of the generated sequence and the index of the reference sequence it is matched to
    """

    # Tokenize the sequences
    generated_sequences = [word_tokenize(seq) for seq in generated_sequences]
    reference_sequences = [word_tokenize(seq) for seq in reference_sequences]

    # Calculate the METEOR scores
    scores = calculate_meteor_scores(generated_sequences, reference_sequences)

    # Convert the scores to a cost matrix (negative scores because Hungarian Algorithm minimizes cost)
    cost_matrix = [[-score for score in row] for row in scores]

    # Apply the Hungarian Algorithm to find the optimal assignment
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # Calculate the total score using the Hungarian algorithm
    total_score = 0
    for i, j in zip(row_ind, col_ind):
        total_score += scores[i][j]

    # Normalize the total score by the number of reference sequences
    total_score = total_score / len(reference_sequences)

    # Best score for each generated question (max score across all references)
    best_scores = [max(row) for row in scores]

    return total_score, best_scores, list(zip(row_ind, col_ind))

for data in new_data:
  generated_sequences = [question['question']+'\n'+question.get('answer', "") for question in data['generatedQuestions'] if question.get("question")]
  reference_sequences = [question['question']+'\n'+question['answer'] for question in data['goldQuestions']]
  if not generated_sequences or not reference_sequences:
    continue
  total_score, best_scores, optimal_matching = hungarian_meteor(generated_sequences, reference_sequences)

  # Print the best METEOR score for each generated question
  for i, score in enumerate(best_scores):
      print(f"Generated Question {i+1}: {''.join(generated_sequences[i])}")
      print(f"Best METEOR Score: {score:.4f}")

  # Print the total score (calculated from Hungarian matching)
  hungarian_meteor_scores.append(total_score)
  print(f"Total METEOR Score (normalized): {total_score:.4f}")

  # Optional: If you want to print the optimal matching from the Hungarian algorithm
  print("Optimal Matching from Hungarian Algorithm:")
  for i, j in optimal_matching:
      print(f"Generated Question {i+1} matched with Reference Question {j+1}")
  print("-" * 50)

# Calculate the average of all total METEOR scores
average_meteor_score = sum(hungarian_meteor_scores) / len(hungarian_meteor_scores) if hungarian_meteor_scores else 0
print(f"\nAverage METEOR Score: {average_meteor_score:.4f}")

Streaming output truncated to the last 5000 lines.
Generated Question 3: What is the nature of the slogans chanted by Hindus in the Maharashtra residential society?
The slogans were in support of Lord Ram.
Best METEOR Score: 0.1240
Total METEOR Score (normalized): 0.1282
Optimal Matching from Hungarian Algorithm:
Generated Question 1 matched with Reference Question 1
Generated Question 2 matched with Reference Question 2
Generated Question 3 matched with Reference Question 3
--------------------------------------------------
Generated Question 1: Is the claim that BJP accuses Naveen Patnaik of saving culprits in panchayat official's death true?
This is a complex political claim.  While the BJP has made this accusation, it is important to investigate the evidence and context to form a well-informed opinion.
Best METEOR Score: 0.2148
Generated Question 2: Has BJP made any accusations against Naveen Patnaik regarding the panchayat official's death?
Yes, BJP has accused Naveen Patnaik of p